# Pra-pemrosesan Data Satelit (NDVI, LST, CHIRPS)

Notebook ini berisi langkah-langkah penggabungan data NDVI, LST, dan CHIRPS menjadi satu file time series gabungan dengan format kolom: `date`, `lon`, `lat`, `LST`, `NDVI`, `CHIRPS`. File akhir bernama `Lembang_Merged_Timeseries.csv`.

In [ ]:
import pandas as pd

In [ ]:
# Load masing-masing file CSV
ndvi = pd.read_csv('NDVI_Monthly_Lembang_Samples_From_L8_Start.csv')
lst = pd.read_csv('LST_Monthly_Lembang_Samples_From_L8_Start.csv')
chirps = pd.read_csv('CHIRPS_Monthly_Lembang_Samples.csv')

In [ ]:
# Ekstrak koordinat dari kolom .geo
import ast

def extract_lon_lat(geo_str):
    coord = ast.literal_eval(geo_str)['coordinates']
    return pd.Series({'lon': coord[0], 'lat': coord[1]})

ndvi[['lon', 'lat']] = ndvi['.geo'].apply(extract_lon_lat)
lst[['lon', 'lat']] = lst['.geo'].apply(extract_lon_lat)
chirps[['lon', 'lat']] = chirps['.geo'].apply(extract_lon_lat)

In [ ]:
# Gabungkan dataframe berdasarkan date, lon, lat
ndvi = ndvi[['date', 'lon', 'lat', 'NDVI']]
lst = lst[['date', 'lon', 'lat', 'LST_C']]
chirps = chirps[['date', 'lon', 'lat', 'CHIRPS_mm']]

# Gabungkan bertahap
merged = pd.merge(ndvi, lst, on=['date', 'lon', 'lat'])
merged = pd.merge(merged, chirps, on=['date', 'lon', 'lat'])

In [ ]:
# Rename kolom
merged = merged.rename(columns={'LST_C': 'LST', 'CHIRPS_mm': 'CHIRPS'})

In [ ]:
# Simpan ke CSV
merged.to_csv('Lembang_Merged_Timeseries.csv', index=False)
merged.head()